# Step 1: Create Private VPC Infrastructure for Production AgentCore

This notebook creates a secure, production-ready VPC infrastructure for hosting Amazon Bedrock AgentCore.

## Architecture Components
- Private VPC with multiple AZs
- Private and public subnets
- Internet Gateway and NAT Gateways
- VPC Endpoints for AWS services
- Security Groups and NACLs

## Prerequisites

Install required dependencies for VPC creation and management.

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

In [ ]:
import boto3
import json
from botocore.exceptions import ClientError
import time

# Initialize AWS clients
ec2 = boto3.client('ec2')
ec2_resource = boto3.resource('ec2')

# Configuration
VPC_CIDR = '10.0.0.0/16'
REGION = boto3.Session().region_name
PROJECT_NAME = 'agentcore-production'

print(f"AWS Region: {REGION}")

## 1. Create VPC

In [ ]:
print(f"Creating VPC infrastructure in region: {REGION}")

# Create VPC
try:
    vpc_response = ec2.create_vpc(
        CidrBlock=VPC_CIDR,
        TagSpecifications=[
            {
                'ResourceType': 'vpc',
                'Tags': [
                    {'Key': 'Name', 'Value': f'{PROJECT_NAME}-vpc'},
                    {'Key': 'Project', 'Value': PROJECT_NAME}
                ]
            }
        ]
    )
    
    vpc_id = vpc_response['Vpc']['VpcId']
    print(f"✅ VPC created: {vpc_id}")
    
    # Wait for VPC to be available
    ec2.get_waiter('vpc_available').wait(VpcIds=[vpc_id])
    
except ClientError as e:
    print(f"❌ Error creating VPC: {e}")
    raise

In [ ]:
# Enable DNS support and DNS hostnames for VPC endpoints
try:
    # Enable DNS support
    ec2.modify_vpc_attribute(
        VpcId=vpc_id,
        EnableDnsSupport={'Value': True}
    )
    
    # Enable DNS hostnames
    ec2.modify_vpc_attribute(
        VpcId=vpc_id,
        EnableDnsHostnames={'Value': True}
    )
    
    print("✅ VPC DNS attributes enabled for VPC endpoints")
    
except ClientError as e:
    print(f"⚠️  Error enabling VPC DNS attributes: {e}")

## 2. Create Subnets

In [ ]:
# Get availability zones
azs = ec2.describe_availability_zones()['AvailabilityZones']
az_names = [az['ZoneName'] for az in azs[:2]]  # Use first 2 AZs

subnets = {}

# Create private subnets
for i, az in enumerate(az_names):
    private_subnet = ec2.create_subnet(
        VpcId=vpc_id,
        CidrBlock=f'10.0.{i+1}.0/24',
        AvailabilityZone=az,
        TagSpecifications=[
            {
                'ResourceType': 'subnet',
                'Tags': [
                    {'Key': 'Name', 'Value': f'{PROJECT_NAME}-private-subnet-{i+1}'},
                    {'Key': 'Type', 'Value': 'Private'},
                    {'Key': 'Project', 'Value': PROJECT_NAME}
                ]
            }
        ]
    )
    subnets[f'private_{i+1}'] = private_subnet['Subnet']['SubnetId']
    print(f"✅ Private subnet {i+1} created: {private_subnet['Subnet']['SubnetId']}")

# Create public subnets
for i, az in enumerate(az_names):
    public_subnet = ec2.create_subnet(
        VpcId=vpc_id,
        CidrBlock=f'10.0.{i+10}.0/24',
        AvailabilityZone=az,
        TagSpecifications=[
            {
                'ResourceType': 'subnet',
                'Tags': [
                    {'Key': 'Name', 'Value': f'{PROJECT_NAME}-public-subnet-{i+1}'},
                    {'Key': 'Type', 'Value': 'Public'},
                    {'Key': 'Project', 'Value': PROJECT_NAME}
                ]
            }
        ]
    )
    subnets[f'public_{i+1}'] = public_subnet['Subnet']['SubnetId']
    print(f"✅ Public subnet {i+1} created: {public_subnet['Subnet']['SubnetId']}")

## 3. Create Internet Gateway and NAT Gateways

### 3.1 Internet Gateway

In [ ]:
# Create Internet Gateway
igw_response = ec2.create_internet_gateway(
    TagSpecifications=[
        {
            'ResourceType': 'internet-gateway',
            'Tags': [
                {'Key': 'Name', 'Value': f'{PROJECT_NAME}-igw'},
                {'Key': 'Project', 'Value': PROJECT_NAME}
            ]
        }
    ]
)
igw_id = igw_response['InternetGateway']['InternetGatewayId']

# Attach IGW to VPC
ec2.attach_internet_gateway(InternetGatewayId=igw_id, VpcId=vpc_id)
print(f"✅ Internet Gateway created and attached: {igw_id}")

### 3.2 Nat Gateway

In productive workloads, we should consider redudance, that's why NAT Gateway is being created into 2 Availability Zones.

If you want to test in a non-productive environment, you can uncomment `break` command in the following cell.

In [ ]:
# Create Elastic IPs for NAT Gateways
nat_gateways = {}
for i in range(len(az_names)):
    # Allocate Elastic IP
    eip_response = ec2.allocate_address(
        Domain='vpc',
        TagSpecifications=[
            {
                'ResourceType': 'elastic-ip',
                'Tags': [
                    {'Key': 'Name', 'Value': f'{PROJECT_NAME}-nat-eip-{i+1}'},
                    {'Key': 'Project', 'Value': PROJECT_NAME}
                ]
            }
        ]
    )
    
    # Create NAT Gateway
    nat_response = ec2.create_nat_gateway(
        SubnetId=subnets[f'public_{i+1}'],
        AllocationId=eip_response['AllocationId']
    )
    
    nat_gateways[f'nat_{i+1}'] = nat_response['NatGateway']['NatGatewayId']
    print(f"✅ NAT Gateway {i+1} created: {nat_response['NatGateway']['NatGatewayId']}")
    #break # uncomment to create NAT in only 1 subnet

# Wait for NAT Gateways to be available
print("⏳ Waiting for NAT Gateways to be available...")
time.sleep(60)  # NAT Gateways take time to become available

## 4. Create Route Tables

If in previous step, you've chosen to create NAT into only 1 AZ, please uncomment `break` block in following cell:

In [ ]:
# Create public route table
public_rt_response = ec2.create_route_table(
    VpcId=vpc_id,
    TagSpecifications=[
        {
            'ResourceType': 'route-table',
            'Tags': [
                {'Key': 'Name', 'Value': f'{PROJECT_NAME}-public-rt'},
                {'Key': 'Type', 'Value': 'Public'},
                {'Key': 'Project', 'Value': PROJECT_NAME}
            ]
        }
    ]
)
public_rt_id = public_rt_response['RouteTable']['RouteTableId']

# Add route to Internet Gateway
ec2.create_route(
    RouteTableId=public_rt_id,
    DestinationCidrBlock='0.0.0.0/0',
    GatewayId=igw_id
)

# Associate public subnets with public route table
for i in range(len(az_names)):
    ec2.associate_route_table(
        RouteTableId=public_rt_id,
        SubnetId=subnets[f'public_{i+1}']
    )

print(f"✅ Public route table created: {public_rt_id}")

# Create private route tables (one per AZ)
private_route_tables = {}
for i in range(len(az_names)):
    private_rt_response = ec2.create_route_table(
        VpcId=vpc_id,
        TagSpecifications=[
            {
                'ResourceType': 'route-table',
                'Tags': [
                    {'Key': 'Name', 'Value': f'{PROJECT_NAME}-private-rt-{i+1}'},
                    {'Key': 'Type', 'Value': 'Private'},
                    {'Key': 'Project', 'Value': PROJECT_NAME}
                ]
            }
        ]
    )
    
    private_rt_id = private_rt_response['RouteTable']['RouteTableId']
    private_route_tables[f'private_rt_{i+1}'] = private_rt_id
    
    # Add route to NAT Gateway
    ec2.create_route(
        RouteTableId=private_rt_id,
        DestinationCidrBlock='0.0.0.0/0',
        NatGatewayId=nat_gateways[f'nat_{i+1}']
    )
    
    # Associate private subnet with private route table
    ec2.associate_route_table(
        RouteTableId=private_rt_id,
        SubnetId=subnets[f'private_{i+1}']
    )
    
    print(f"✅ Private route table {i+1} created: {private_rt_id}")
    #break # uncomment to consider only 1 az

## 5. Create VPC Endpoints

In [ ]:
# VPC Endpoints for AWS services
vpc_endpoints = [
    'com.amazonaws.{}.bedrock-runtime'.format(REGION),
    'com.amazonaws.{}.bedrock-agent-runtime'.format(REGION),
    'com.amazonaws.{}.logs'.format(REGION),
    'com.amazonaws.{}.monitoring'.format(REGION),
    'com.amazonaws.{}.sts'.format(REGION)
]

# Create security group for VPC endpoints
endpoint_sg_response = ec2.create_security_group(
    GroupName=f'{PROJECT_NAME}-vpc-endpoints-sg',
    Description='Security group for VPC endpoints',
    VpcId=vpc_id,
    TagSpecifications=[
        {
            'ResourceType': 'security-group',
            'Tags': [
                {'Key': 'Name', 'Value': f'{PROJECT_NAME}-vpc-endpoints-sg'},
                {'Key': 'Project', 'Value': PROJECT_NAME}
            ]
        }
    ]
)
endpoint_sg_id = endpoint_sg_response['GroupId']

# Add inbound rule for HTTPS
ec2.authorize_security_group_ingress(
    GroupId=endpoint_sg_id,
    IpPermissions=[
        {
            'IpProtocol': 'tcp',
            'FromPort': 443,
            'ToPort': 443,
            'IpRanges': [{'CidrIp': VPC_CIDR}]
        }
    ]
)

print(f"✅ VPC Endpoints security group created: {endpoint_sg_id}")

# Create VPC endpoints
private_subnet_ids = [subnets[f'private_{i+1}'] for i in range(len(az_names))]

for service_name in vpc_endpoints:
    try:
        endpoint_response = ec2.create_vpc_endpoint(
            VpcId=vpc_id,
            ServiceName=service_name,
            VpcEndpointType='Interface',
            SubnetIds=private_subnet_ids,
            SecurityGroupIds=[endpoint_sg_id],
            PrivateDnsEnabled=True,
            TagSpecifications=[
                {
                    'ResourceType': 'vpc-endpoint',
                    'Tags': [
                        {'Key': 'Name', 'Value': f'{PROJECT_NAME}-{service_name.split(".")[-1]}-endpoint'},
                        {'Key': 'Project', 'Value': PROJECT_NAME}
                    ]
                }
            ]
        )
        print(f"✅ VPC Endpoint created for {service_name}")
    except ClientError as e:
        if 'InvalidServiceName' in str(e):
            print(f"⚠️  Service {service_name} not available in {REGION}")
        else:
            print(f"❌ Error creating endpoint for {service_name}: {e}")

## 6. Create Security Groups for AgentCore

In [ ]:
# Create security group for AgentCore Runtime
agentcore_sg_response = ec2.create_security_group(
    GroupName=f'{PROJECT_NAME}-agentcore-sg',
    Description='Security group for AgentCore Runtime',
    VpcId=vpc_id,
    TagSpecifications=[
        {
            'ResourceType': 'security-group',
            'Tags': [
                {'Key': 'Name', 'Value': f'{PROJECT_NAME}-agentcore-sg'},
                {'Key': 'Project', 'Value': PROJECT_NAME}
            ]
        }
    ]
)
agentcore_sg_id = agentcore_sg_response['GroupId']

# Add inbound rules for AgentCore
ec2.authorize_security_group_ingress(
    GroupId=agentcore_sg_id,
    IpPermissions=[
        {
            'IpProtocol': 'tcp',
            'FromPort': 8080,
            'ToPort': 8080,
            'IpRanges': [{'CidrIp': VPC_CIDR}]
        },
        {
            'IpProtocol': 'tcp',
            'FromPort': 443,
            'ToPort': 443,
            'IpRanges': [{'CidrIp': VPC_CIDR}]
        }
    ]
)

print(f"✅ AgentCore security group created: {agentcore_sg_id}")

## 7. Save Configuration

In [ ]:
# Save configuration for next notebook
config = {
    'vpc_id': vpc_id,
    'subnets': subnets,
    'security_groups': {
        'agentcore_sg_id': agentcore_sg_id,
        'endpoint_sg_id': endpoint_sg_id
    },
    'nat_gateways': nat_gateways,
    'route_tables': private_route_tables,
    'region': REGION,
    'project_name': PROJECT_NAME
}

with open('vpc_config.json', 'w') as f:
    json.dump(config, f, indent=2)

print("✅ Configuration saved to vpc_config.json")
print("\n🎉 VPC infrastructure created successfully!")
print(f"VPC ID: {vpc_id}")
print(f"Private Subnets: {[subnets[f'private_{i+1}'] for i in range(len(az_names))]}")
print(f"AgentCore Security Group: {agentcore_sg_id}")